In [21]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns

In [38]:
# returns a distance value between two series
# e.g. s1 = sgmd(eigen1) & s2 = relu(eigen1)
# ranging from k0 to k9 for 1 epoch
def get_distance_epoch_value(s1, s2):
    s1 = s1.abs()
    s2 = s2.abs()
    
    dist = s1 - s2
    
    max_dist = max(dist)
    max_norm = np.linalg.norm([max_dist] * len(s1))
    
    return np.linalg.norm(dist) / max_norm

In [39]:
# e.g. activator = 'sgmd'
# returns a list of kernel values for an eigen number for 1 epoch
def build_series_1epoch(dataset, activator, eig, e_no):
    df_cnn_relu0 = pd.read_csv(dataset + "/results/" + activator + "/cnn_K0.csv")
    df_cnn_relu1 = pd.read_csv(dataset + "/results/" + activator + "/cnn_K1.csv")
    df_cnn_relu2 = pd.read_csv(dataset + "/results/" + activator + "/cnn_K2.csv")
    df_cnn_relu3 = pd.read_csv(dataset + "/results/" + activator + "/cnn_K3.csv")
    df_cnn_relu4 = pd.read_csv(dataset + "/results/" + activator + "/cnn_K4.csv")
    df_cnn_relu5 = pd.read_csv(dataset + "/results/" + activator + "/cnn_K5.csv")
    df_cnn_relu6 = pd.read_csv(dataset + "/results/" + activator + "/cnn_K6.csv")
    df_cnn_relu7 = pd.read_csv(dataset + "/results/" + activator + "/cnn_K7.csv")
    df_cnn_relu8 = pd.read_csv(dataset + "/results/" + activator + "/cnn_K8.csv")
    df_cnn_relu9 = pd.read_csv(dataset + "/results/" + activator + "/cnn_K9.csv")

    df_1 =   pd.Series([df_cnn_relu0.iloc[e_no][eig-1], df_cnn_relu1.iloc[e_no][eig-1], 
              df_cnn_relu2.iloc[e_no][eig-1], df_cnn_relu3.iloc[e_no][eig-1], 
              df_cnn_relu4.iloc[e_no][eig-1], df_cnn_relu5.iloc[e_no][eig-1],
              df_cnn_relu6.iloc[e_no][eig-1], df_cnn_relu7.iloc[e_no][eig-1], 
              df_cnn_relu8.iloc[e_no][eig-1], df_cnn_relu9.iloc[e_no][eig-1]])
    
    return df_1

In [40]:
# e.g. fcn1 = 'sgmd' & fcn2 = 'relu'
# returns a list of distances between 2 activators over all epochs for 1 eigen number
def get_distance_1eig_epoch_list(dataset, fcn1, fcn2, eig):
    
    epoch_list = []
    for epoch in range(0, 10):
        s1 = build_series_1epoch(dataset, fcn1, eig, epoch)
        s2 = build_series_1epoch(dataset, fcn2, eig, epoch)
        epoch_list.append(get_distance_epoch_value(s1, s2))
        
    max_norm = max(epoch_list)
    for i in range(len(epoch_list)):
        epoch_list[i] = epoch_list[i] / max_norm
        
    return epoch_list
    

In [68]:
def get_err(dataset, fcn1, fcn2):
    e1 = pd.read_csv(dataset + "/results/" + fcn1 + "/cnn_test_loss.csv")
    e2 = pd.read_csv(dataset + "/results/" + fcn2 + "/cnn_test_loss.csv")
    
    err = ((e1['0'] - e2['0']).abs() + 1) / (e1['0'] + e2['0'] + 1)
    
#     max_norm = max(err)
#     for i in range(len(err)):
#         err[i] = err[i] / max_norm
    
    return err

In [69]:
def get_corr_to_error(dataset, fcn1, fcn2, eig):
    
    dists = get_distance_1eig_epoch_list(dataset, fcn1, fcn2, eig)
    err = get_err(dataset, fcn1, fcn2)
    
    return np.corrcoef(dists, err)[0][1]
    

In [70]:
get_corr_to_error('mnist', 'tanh', 'relu', 5)

0.10669355327738629